# Import e funzioni ausiliarie

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from tensorflow import keras as k

import tensorflow as tf
import pandas as pd
import numpy as np

# %pip install d2l==1.0.0a1.post0
from d2l import tensorflow as d2l


In [2]:
data = pd.read_csv('./input/dataset.csv')
# dataImputed = pd.read_csv('./input/datasetImputed.csv')

In [3]:
# è stata riscritta una funzione di reshape inutile 
# e.g. (a_prev=x_train, season=trn_ssn)
def reshape_to_inputshape(a_prev,season):
    totalMatches = len(season)*38
    input_step = int(a_prev.shape[0]/totalMatches)
    prev_f = a_prev.shape[1]
    return np.reshape(a_prev, (totalMatches, input_step, prev_f))

In [4]:
def encode(x):
    if x == 0:
        v = [1,0,0]
    elif x == 1:
        v = [0,1,0]
    elif x == 2:
        v = [0,0,1]
    return v

def encode_list(l):
    r = []
    for i in l:
        r.append(encode(i))
    return r

In [5]:
def report(model, x_train, y_train, x_test, y_test, encoder):
    # TRAIN
    y_pred_train = model.predict(x_train)
    y_predm_train = np.asarray(y_pred_train)
    y_predm_train = np.argmax(y_predm_train, axis=2)
    y_predm_train = encode_list(y_predm_train.reshape(-1))
    y_predm_train = np.array(y_predm_train)

    y_trainm = np.argmax(y_train, axis = 2)
    y_trainm = encode_list(y_trainm.reshape(-1))
    y_trainm = np.array(y_trainm)

    # Inverse One-hot transform - TRAIN
    y_predm_train = encoder.inverse_transform(y_predm_train)
    y_trainm = encoder.inverse_transform(y_trainm)

    # TEST
    y_pred = model.predict(x_test)
    y_predm = np.asarray(y_pred)
    y_predm = np.argmax(y_predm, axis=2)
    y_predm = encode_list(y_predm.reshape(-1))
    y_predm = np.array(y_predm)
    
    y_testm = np.argmax(y_test, axis=2)
    y_testm = encode_list(y_testm.reshape(-1))
    y_testm = np.array(y_testm)

    # Inverse One-hot transform - TEST
    y_predm = encoder.inverse_transform(y_predm)
    y_testm = encoder.inverse_transform(y_testm)

    #Model Metrics test
    print("TEST REPORT")
    print(classification_report(y_testm, y_predm, digits=3))
    print("--------------------------------------------------")

    #Model Metrics train
    print("TRAINING REPORT")
    print(classification_report(y_trainm, y_predm_train, digits=3))
    print("--------------------------------------------------")

# Preprocessamento dati

In [6]:
features = ['HomeTeam', 'AwayTeam', 
            'HTeamEloScore', 'ATeamEloScore', 
            'HTdaysSinceLastMatch', 'ATdaysSinceLastMatch', 
            'HTW_rate', 'ATW_rate', 'ATD_rate', 'HTD_rate', 
            '7_HTW_rate', '12_HTW_rate', '7_ATW_rate', '12_ATW_rate', 
            '7_HTD_rate', '12_HTD_rate', '7_ATD_rate', '12_ATD_rate',
            '7_HTL_rate', '12_HTL_rate', '7_ATL_rate', '12_ATL_rate',
            '5_HTHW_rate', '5_ATAW_rate']

X = pd.get_dummies(data[features])

# Se non cambiamo nulla il OneHotEncoder assegna:
# A -> 1 0 0
# D -> 0 1 0
# H -> 0 0 1
y = data[['FTR']].to_numpy().ravel().reshape(-1, 1)
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)


In [7]:
trn_ssn = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
trn_ssn_len = len(trn_ssn)
tst_ssn = [2016,2017,2018] 
tst_ssn_len = len(tst_ssn)

test_size = float(tst_ssn_len)/(tst_ssn_len+trn_ssn_len)

#Split X and Y into training and Test Sets
x_train, x_test, y_train, y_test = train_test_split(X_imputed, y, shuffle=False, test_size=test_size)

# RandomForestClassifier

In [8]:
forest = RandomForestClassifier(n_estimators=2, random_state=2)
forest = forest.fit(x_train, y_train)


#Forest Model Metrics
print("Forest Classifier")
print("Train Score: ", forest.score(x_train, y_train))
print("Test Score: ", forest.score(x_test, y_test))

Forest Classifier
Train Score:  0.6302631578947369
Test Score:  0.22017543859649122


In [9]:
n = 10
m = 5
max_depth = 10
forests = []
grid = [{"n_estimators": list(range(1, n)), "random_state": list(range(0, m)), "max_depth": list(range(1, max_depth))}]
gridSearch = GridSearchCV(RandomForestClassifier(), param_grid=grid, n_jobs=10, return_train_score=True)
gridSearch.fit(x_train, y_train)

print("Forest Classifiers Best Score: ", gridSearch.best_score_)
print("Forest Classifiers Best Params: ", gridSearch.best_params_)
print("Forest Classifiers Best Params: ", gridSearch.best_estimator_)

Forest Classifiers Best Score:  0.4305921052631579
Forest Classifiers Best Params:  {'max_depth': 8, 'n_estimators': 1, 'random_state': 1}
Forest Classifiers Best Params:  RandomForestClassifier(max_depth=8, n_estimators=1, random_state=1)


# Neural Network

In [10]:
y_nn = ((data[['ordinalHR']]).to_numpy()*2)
#Split X and Y into training and Test Sets
# y_nn = OneHotEncoder(sparse=True).fit_transform(y_nn)
x_train_nn, x_test_nn, y_train_nn, y_test_nn = train_test_split(X_imputed, y_nn, shuffle=True)

In [11]:
nn = k.models.Sequential([
    k.layers.Flatten(),
    k.layers.Dense(41, activation='relu'),
    k.layers.Dense(75, activation='relu'),
    k.layers.Dropout(0.3),
    k.layers.Dense(3, activation='softmax'),
])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-11-13 15:14:27.384024: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-13 15:14:27.384148: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
learning_rate=0.01

nn.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=k.optimizers.Adam(learning_rate),
    metrics=['accuracy']
)
nn(x_train_nn)
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (3135, 94)                0         
                                                                 
 dense (Dense)               (3135, 41)                3895      
                                                                 
 dense_1 (Dense)             (3135, 75)                3150      
                                                                 
 dropout (Dropout)           (3135, 75)                0         
                                                                 
 dense_2 (Dense)             (3135, 3)                 228       
                                                                 
Total params: 7,273
Trainable params: 7,273
Non-trainable params: 0
_________________________________________________________________


In [13]:
epochs=10
batch_size=50
nn.fit(x_train_nn, y_train_nn, epochs=epochs, batch_size=batch_size)

Epoch 1/10


2022-11-13 15:14:27.610407: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-13 15:14:27.808556: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 1s 7ms/step - loss: 2.2527 - accuracy: 0.4931
Epoch 2/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9570 - accuracy: 0.5617
Epoch 3/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9497 - accuracy: 0.5662
Epoch 4/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9446 - accuracy: 0.5630
Epoch 5/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9424 - accuracy: 0.5608
Epoch 6/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9377 - accuracy: 0.5598
Epoch 7/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9459 - accuracy: 0.5675
Epoch 8/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9434 - accuracy: 0.5592
Epoch 9/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9458 - accuracy: 0.5569
Epoch 10/10
63/63 [==============================] - 0s 6ms/step - loss: 0.9418 - accuracy: 0.5553


In [14]:
print(nn.evaluate(x_test_nn, y_test_nn))

23/33 [===================>..........] - ETA: 0s - loss: 0.9284 - accuracy: 0.5639

2022-11-13 15:14:32.040678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.9098 - accuracy: 0.5780
[0.9098198413848877, 0.5779904127120972]


# Lstm preprocessing

In [40]:
#Setup XY to have 10 game steps
x_train_lstm = reshape_to_inputshape(x_train,trn_ssn)
y_train_lstm = reshape_to_inputshape(y_train,trn_ssn)
# y_train = np.moveaxis(y_train, 0, 1)

x_test_lstm = reshape_to_inputshape(x_test,tst_ssn)
y_test_lstm = reshape_to_inputshape(y_test,tst_ssn)
# y_test = np.moveaxis(y_test, 0, 1)

Tx = x_train_lstm.shape[1] #Time steps
print(Tx)
Ty = y_train_lstm.shape[1] #Time Steps
print(Ty)

num_features = x_train_lstm.shape[2] #Features per step
inputs = tf.keras.Input(shape=(Tx, num_features))

10
10


# LSTM

In [16]:
model_path = './models/lstm/'
# Check se il modello è già salvato
model = k.models.load_model(model_path)
if model == None:
    model = k.models.Sequential(
        [
            k.layers.LSTM(64, return_sequences=True),
            k.layers.Dropout(0.4),
            k.layers.Dense(1000, activation="relu"),
            k.layers.Dropout(0.3),
            k.layers.Dense(250, activation="relu"),
            k.layers.Dropout(0.2),
            k.layers.Dense(3, activation="softmax")
        ]
    )

    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=["accuracy"]
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

    model.fit(x_train_lstm, y_train_lstm, epochs=10, callbacks=[callback])
    model.save('./models/lstm')

In [17]:
model.evaluate(x_test_lstm, y_test_lstm)
report(model, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

1/4 [======>.......................] - ETA: 1s - loss: 0.9114 - accuracy: 0.5844

2022-11-13 15:14:35.926440: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:14:35.992713: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 7/10 [====================>.........] - ETA: 0s

2022-11-13 15:14:36.381910: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:14:36.431515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 0s 16ms/step
TEST REPORT
              precision    recall  f1-score   support

           A      0.577     0.643     0.608       345
           D      0.287     0.114     0.163       254
           H      0.641     0.774     0.701       541

    accuracy                          0.588      1140
   macro avg      0.501     0.511     0.491      1140
weighted avg      0.543     0.588     0.553      1140

--------------------------------------------------
TRAINING REPORT
              precision    recall  f1-score   support

           A      0.515     0.640     0.571       867
           D      0.419     0.126     0.194       783
           H      0.625     0.776     0.693      1390

    accuracy                          0.570      3040
   macro avg      0.520     0.514     0.486      3040
weighted avg      0.541     0.570     0.529      3040

--------------------------------------------------


## 2 LSTM concatenate + dense e dropout in sequenza

In [18]:
inputs = k.layers.Input(shape=(10,94))
lstm1 = k.layers.LSTM(100, return_sequences=True, activation="relu")(inputs)
lstm2 = k.layers.LSTM(50, return_sequences=True, activation="relu")(inputs)
concateneted = k.layers.Concatenate()([
    lstm1,
    lstm2
])

out = k.layers.Dropout(0.5)(concateneted)
out = k.layers.Dense(1000, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(3, activation="softmax")(out)

model = k.models.Model(inputs=inputs, outputs=out)

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10, 94)]     0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 10, 100)      78000       ['input_2[0][0]']                
                                                                                                  
 lstm_1 (LSTM)                  (None, 10, 50)       29000       ['input_2[0][0]']                
                                                                                                  
 concatenate (Concatenate)      (None, 10, 150)      0           ['lstm[0][0]',                   
                                                                  'lstm_1[0][0]']             

In [21]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

Epoch 1/500


2022-11-13 15:15:34.303184: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 4s 235ms/step - loss: 1.2438 - accuracy: 0.4954
Epoch 2/500
10/10 [==============================] - 2s 218ms/step - loss: 1.1790 - accuracy: 0.5026
Epoch 3/500
10/10 [==============================] - 2s 213ms/step - loss: 1.1642 - accuracy: 0.5049
Epoch 4/500
10/10 [==============================] - 2s 210ms/step - loss: 1.1169 - accuracy: 0.5181
Epoch 5/500
10/10 [==============================] - 2s 213ms/step - loss: 1.1297 - accuracy: 0.5201
Epoch 6/500
10/10 [==============================] - 2s 240ms/step - loss: 1.1016 - accuracy: 0.5188
Epoch 7/500
10/10 [==============================] - 2s 244ms/step - loss: 1.1094 - accuracy: 0.5211
Epoch 8/500
10/10 [==============================] - 2s 238ms/step - loss: 1.0991 - accuracy: 0.5102
Epoch 9/500
10/10 [==============================] - 2s 230ms/step - loss: 1.0866 - accuracy: 0.5224
Epoch 10/500
10/10 [==============================] - 2s 222ms/step - loss: 1.0585 - accuracy: 0.5240
E

In [22]:
model.evaluate(x_test_lstm, y_test_lstm)
report(model, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

2022-11-13 15:16:36.444582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 1/10 [==>...........................] - ETA: 4s

2022-11-13 15:16:37.129887: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 0s 53ms/step
TEST REPORT
              precision    recall  f1-score   support

           A      0.588     0.620     0.604       345
           D      0.297     0.228     0.258       254
           H      0.663     0.712     0.686       541

    accuracy                          0.576      1140
   macro avg      0.516     0.520     0.516      1140
weighted avg      0.559     0.576     0.566      1140

--------------------------------------------------
TRAINING REPORT
              precision    recall  f1-score   support

           A      0.527     0.582     0.553       867
           D      0.400     0.245     0.304       783
           H      0.644     0.742     0.689      1390

    accuracy                          0.568      3040
   macro avg      0.524     0.523     0.516      3040
weighted avg      0.548     0.568     0.551      3040

--------------------------------------------------


### Fit
loss: 0.9373 - accuracy: 0.5609
### Evaluate
loss: 0.9192 - accuracy: 0.5930
### Classification report

| | precision | recall | f1-score | support |
| - | ----------- | ------ | -------- | ------- |
| A | 0.609 | 0.614 | 0.612 | 345 |
| D | 0.321 | 0.201 | 0.247 | 254 |
| H | 0.652 | 0.763 | 0.704 | 541 |
| accuracy | |  | 0.593 | 1140 |
| macro avg | 0.527 | 0.526 | 0.521 | 1140 |
| weighted avg | 0.565 | 0.593 | 0.574 | 1140 |

## LSTM in parallelo

In [23]:
inputs = k.layers.Input(shape=(10,94))
x = []
for t in range(Tx):
    module = k.layers.LSTM((t+1)*10, return_sequences=True)(inputs)
    module = k.layers.Dropout(0.7)(module)
    x.append(module)
x = k.layers.Concatenate()(x)

out = k.layers.Dense(1000, activation="relu")(x)
out = k.layers.Dropout(0.7)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dense(3, activation="softmax")(out)

txLstm = k.models.Model(inputs=inputs, outputs=out)

In [24]:
txLstm.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 10, 94)]     0           []                               
                                                                                                  
 lstm_2 (LSTM)                  (None, 10, 10)       4200        ['input_3[0][0]']                
                                                                                                  
 lstm_3 (LSTM)                  (None, 10, 20)       9200        ['input_3[0][0]']                
                                                                                                  
 lstm_4 (LSTM)                  (None, 10, 30)       15000       ['input_3[0][0]']                
                                                                                            

In [25]:
txLstm.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

txLstm.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback], batch_size=16)

Epoch 1/500


2022-11-13 15:16:44.693387: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:16:45.475374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:16:45.493243: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:16:45.511808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:16:45.536154: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:16:45.554216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:16:45.586264: I tensorflow/core/grappler/optimizers/cust

19/19 [==============================] - 10s 129ms/step - loss: 1.2090 - accuracy: 0.4023
Epoch 2/500
19/19 [==============================] - 2s 80ms/step - loss: 1.0827 - accuracy: 0.4750
Epoch 3/500
19/19 [==============================] - 1s 72ms/step - loss: 1.0325 - accuracy: 0.5220
Epoch 4/500
19/19 [==============================] - 1s 66ms/step - loss: 1.0178 - accuracy: 0.5401
Epoch 5/500
19/19 [==============================] - 1s 66ms/step - loss: 0.9966 - accuracy: 0.5414
Epoch 6/500
19/19 [==============================] - 1s 65ms/step - loss: 0.9933 - accuracy: 0.5418
Epoch 7/500
19/19 [==============================] - 1s 65ms/step - loss: 0.9754 - accuracy: 0.5382
Epoch 8/500
19/19 [==============================] - 1s 65ms/step - loss: 0.9601 - accuracy: 0.5618
Epoch 9/500
19/19 [==============================] - 1s 63ms/step - loss: 0.9468 - accuracy: 0.5625
Epoch 10/500
19/19 [==============================] - 1s 70ms/step - loss: 0.9412 - accuracy: 0.5691
Epoch 11/

In [32]:
txLstm.evaluate(y_test_lstm, y_test_lstm)
report(txLstm, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

4/4 [==============================] - 0s 112ms/step - loss: 1.0929 - accuracy: 0.4746


2022-11-13 15:20:03.319232: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:20:03.600516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:20:03.600591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:20:03.600730: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:20:03.600898: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:20:03.600943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:20:03.601022: I tensorflow/core/grappler/optimizers/cust

4/4 [==============================] - 0s 126ms/step
TEST REPORT
              precision    recall  f1-score   support

           A      0.637     0.464     0.537       345
           D      0.270     0.303     0.286       254
           H      0.659     0.736     0.695       541

    accuracy                          0.557      1140
   macro avg      0.522     0.501     0.506      1140
weighted avg      0.566     0.557     0.556      1140

--------------------------------------------------
TRAINING REPORT
              precision    recall  f1-score   support

           A      0.572     0.467     0.514       867
           D      0.376     0.336     0.355       783
           H      0.645     0.758     0.697      1390

    accuracy                          0.566      3040
   macro avg      0.531     0.520     0.522      3040
weighted avg      0.555     0.566     0.557      3040

--------------------------------------------------


# GRU

In [28]:
gru = k.models.Sequential([
    k.layers.GRU(100, return_sequences=True, activation="relu"),
    k.layers.Dropout(0.4),
    k.layers.Dense(1000, activation="relu"),
    k.layers.Dropout(0.3),
    k.layers.Dense(250, activation="relu"),
    k.layers.Dropout(0.2),
    k.layers.Dense(3, activation="softmax")
])

In [29]:
gru(x_train_lstm)
gru.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (304, 10, 100)            58800     
                                                                 
 dropout_15 (Dropout)        (304, 10, 100)            0         
                                                                 
 dense_9 (Dense)             (304, 10, 1000)           101000    
                                                                 
 dropout_16 (Dropout)        (304, 10, 1000)           0         
                                                                 
 dense_10 (Dense)            (304, 10, 250)            250250    
                                                                 
 dropout_17 (Dropout)        (304, 10, 250)            0         
                                                                 
 dense_11 (Dense)            (304, 10, 3)             

In [30]:
gru.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

gru.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

Epoch 1/500


2022-11-13 15:17:21.348375: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 3s 191ms/step - loss: 2.3407 - accuracy: 0.3711
Epoch 2/500
10/10 [==============================] - 2s 161ms/step - loss: 1.9536 - accuracy: 0.4411
Epoch 3/500
10/10 [==============================] - 2s 159ms/step - loss: 1.6842 - accuracy: 0.4428
Epoch 4/500
10/10 [==============================] - 1s 150ms/step - loss: 1.5865 - accuracy: 0.4510
Epoch 5/500
10/10 [==============================] - 1s 151ms/step - loss: 1.4738 - accuracy: 0.4549
Epoch 6/500
10/10 [==============================] - 1s 146ms/step - loss: 1.4370 - accuracy: 0.4497
Epoch 7/500
10/10 [==============================] - 2s 153ms/step - loss: 1.3532 - accuracy: 0.4553
Epoch 8/500
10/10 [==============================] - 2s 160ms/step - loss: 1.2920 - accuracy: 0.4576
Epoch 9/500
10/10 [==============================] - 2s 171ms/step - loss: 1.2556 - accuracy: 0.4813
Epoch 10/500
10/10 [==============================] - 2s 153ms/step - loss: 1.2352 - accuracy: 0.4628
E

In [31]:
gru.evaluate(x_test_lstm, y_test_lstm)
report(gru, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

2022-11-13 15:18:52.873423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 1/10 [==>...........................] - ETA: 2s

2022-11-13 15:18:53.437469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 0s 52ms/step
TEST REPORT
              precision    recall  f1-score   support

           A      0.547     0.643     0.591       345
           D      0.322     0.110     0.164       254
           H      0.629     0.752     0.685       541

    accuracy                          0.576      1140
   macro avg      0.499     0.502     0.480      1140
weighted avg      0.536     0.576     0.541      1140

--------------------------------------------------
TRAINING REPORT
              precision    recall  f1-score   support

           A      0.516     0.603     0.556       867
           D      0.513     0.156     0.239       783
           H      0.625     0.804     0.703      1390

    accuracy                          0.580      3040
   macro avg      0.551     0.521     0.499      3040
weighted avg      0.565     0.580     0.542      3040

--------------------------------------------------


# Loro modello

In [43]:
#Create and Setup Model
fbmodel = tf.keras.Sequential()
inputs = tf.keras.Input(shape=(Tx, num_features))
outputs = []
for t in range(Ty):
    x = tf.keras.layers.Lambda(lambda z: inputs[:, t,:])(inputs)

    x = tf.keras.layers.Reshape((1, num_features))(x)
    
    x = tf.keras.layers.LSTM(units=16, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-1))(x)

    x = tf.keras.layers.Dropout(rate=0.8)(x)
    
    out = tf.keras.layers.Dense(3, activation='softmax')(x)
    
    outputs.append(out)
    
fbmodel = tf.keras.Model(inputs=inputs, outputs=outputs)
fbmodel.summary()

fbmodel.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001)
#    ,metrics=[tf.keras.metrics.Accuracy()]
)

ValueError: Input 0 of layer "lstm_318" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 16)

In [ ]:
#Train Model
history = fbmodel.fit(
    x_train, list(y_train),
    epochs=100,
    batch_size=64,
    #validation_split=0.272727,
    #show = epoch%100==0,
    verbose=1,
    shuffle=False,
    
)